In [51]:
import pymatgen as pmg
from pymatgen.io.vasp.outputs import Outcar
from pymatgen.electronic_structure.core import Spin
import numpy as np

from os.path import expanduser
home = expanduser('~')

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")


def format_ladder(orbitals, occu_matrix, spin=None):
    ladder = np.array(sorted(zip(np.diag(occu_matrix), orbitals), reverse=True))
    occu_ladder = ladder[:,0]
    orb_ladder = ladder[:,1]
    if spin == 1:
        row1 = '/\ Orbital:   '
        row2 = '/\ Occupation:'
    elif spin == -1:
        row1 = '\/ Orbital:   '
        row2 = '\/ Occupation:'
    for i in range(len(occu_ladder)):
        row1 = row1 + f'\t{orb_ladder[i]}'
        row2 = row2 + f'\t{occu_ladder[i]}'
    return row1 + '\n' + row2


def display_occu(odm, orbitals, order=None, print_ladder=True):
    up = odm[Spin.up]
    down = odm[Spin.down]
    
    ### Write header
    header = '\t'
    if order == None: order = range(5)
    for i in order:
        header = f'{header}{orbitals[i]}\t\t'
    print(header, end='')
    
    ### Write each row
    for i in order:
        print(f'\n{orbitals[i]}', end='\t')
        for j in order:
            entries = [np.round(up[i][j], 2), np.round(down[i][j], 2)]
            for k in range(2):
                if entries[k] == 0:
                    entries[k] = '----'
            print(f'{entries[0]} / {entries[1]}', end='\t')
    print('\n')
    if print_ladder:
        print(format_ladder(orbitals, up, spin=1))
        print()
        print(format_ladder(orbitals, down, spin=-1))
#         up_ladder = zip(np.diag(up), orbitals)
#         up_ladder =  sorted(up_ladder, reverse=True)
#         print(np.array(up_ladder).T)
#         print('\n')
        
#         dn_ladder = zip(np.diag(down), orbitals)
#         dn_ladder =  sorted(dn_ladder, reverse=True)
#         print(np.array(dn_ladder).T)
#         print('\n')
    

def print_occupations(iters, orbitals, n_with_U, order=None, atom_idx=0):
    odms = []
    for i, iter_1 in enumerate(iters):
            label = f'{iter_1}'
            outcar = Outcar(f'{PREFIX}-{label}{SUFFIX}')
            outcar.read_onsite_density_matrices()
            odm = outcar.data['onsite_density_matrices'][-n_with_U:] # we only want to odm from last iter
            odms.append(odm)
            print(f'##### {iter_1} #####')
            display_occu(odm[atom_idx], orbitals, order=order)
            print()
            
            
PREFIX = home + '/Projects/BaCoS2/correct_mag/disp-estruc/OUTCAR'
SUFFIX = ""

In [55]:
### Unrotated
orbitals_fancy = ['$d_{x^2 - y^2}$', '$d_{yz}$', '$d_{z^2 - r^2}$', '$d_{xz}$', '$d_{xy}$']
orbitals = ['d_x2-y2', 'd_yz', 'd_z2-r2', 'd_xz', 'd_xy']
order = [4, 1, 3, 2, 0]

### Rotated to align with VASP default (ligands along xyz)
rot_orbitals_fancy = ['$d_{xy}$', '$d_{yz}$', '$d_{z^2 - r^2}$', '$d_{xz}$', '$d_{x^2 - y^2}$']
rot_orbitals = ['d_xy', 'd_yz', 'd_z2-r2', 'd_xz', 'd_x2-y2']
rot_order =  [0, 1, 3, 2, 4] #ordering is just for cosmetics -- xz near yz

In [67]:
### Final phonons - primitive cell M4 mode
#################################################
PREFIX = home + '/Projects/BaCoS2/correct_mag/disp-estruc/OUTCAR'

ITERS = [1, 2, 3, 4, 5]
prefix = 'U2-M1-disp'

ITERS = [prefix + str(item) for item in ITERS]
for i in range(1):
    print_occupations(ITERS, rot_orbitals,  n_with_U=8, order=rot_order, atom_idx=i)

# print_occupations(ITERS, orbitals,  n_with_U=8, order=order, atom_idx=0)

##### U2-M1-disp1 #####
	d_xy		d_yz		d_xz		d_z2-r2		d_x2-y2		
d_xy	0.93 / 0.32	---- / ----	---- / ----	---- / ----	---- / ----	
d_yz	---- / ----	0.94 / 0.26	---- / 0.05	---- / ----	---- / ----	
d_xz	---- / ----	---- / 0.05	0.93 / 0.89	---- / ----	---- / ----	
d_z2-r2	---- / ----	---- / ----	---- / ----	0.93 / 0.34	---- / 0.2	
d_x2-y2	---- / ----	---- / ----	---- / ----	---- / 0.2	0.93 / 0.84	

/\ Orbital:   	d_yz	d_x2-y2	d_z2-r2	d_xy	d_xz
/\ Occupation:	0.9407	0.9327	0.9313	0.93	0.9278

\/ Orbital:   	d_xz	d_x2-y2	d_z2-r2	d_xy	d_yz
\/ Occupation:	0.8893	0.8364	0.3405	0.3233	0.2612

##### U2-M1-disp2 #####
	d_xy		d_yz		d_xz		d_z2-r2		d_x2-y2		
d_xy	0.93 / 0.32	---- / ----	---- / ----	---- / ----	---- / 0.01	
d_yz	---- / ----	0.94 / 0.25	---- / 0.03	---- / ----	---- / ----	
d_xz	---- / ----	---- / 0.03	0.93 / 0.89	---- / ----	---- / ----	
d_z2-r2	---- / ----	---- / ----	---- / ----	0.93 / 0.34	---- / 0.21	
d_x2-y2	---- / 0.01	---- / ----	---- / ----	---- / 0.21	0.93 / 0.83	

/\ Orbital: 

In [63]:
### Final phonons - primitive cell M4 mode
#################################################
PREFIX = home + '/Projects/BaCoS2/correct_mag/disp-estruc/OUTCAR'

ITERS = [0]
prefix = 'U2-M1-disp'

ITERS = [prefix + str(item) for item in ITERS]
for i in range(8):
    print_occupations(ITERS, rot_orbitals,  n_with_U=8, order=rot_order, atom_idx=i)

# print_occupations(ITERS, orbitals,  n_with_U=8, order=order, atom_idx=0)

##### U2-M1-disp0 #####
	d_xy		d_yz		d_xz		d_z2-r2		d_x2-y2		
d_xy	0.92 / 0.31	---- / ----	---- / ----	---- / ----	---- / ----	
d_yz	---- / ----	0.94 / 0.22	---- / ----	---- / ----	---- / ----	
d_xz	---- / ----	---- / ----	0.92 / 0.87	---- / ----	---- / ----	
d_z2-r2	---- / ----	---- / ----	---- / ----	0.92 / 0.42	---- / 0.16	
d_x2-y2	---- / ----	---- / ----	---- / ----	---- / 0.16	0.93 / 0.86	

/\ Orbital:   	d_yz	d_x2-y2	d_xz	d_z2-r2	d_xy
/\ Occupation:	0.9403	0.9322	0.9249	0.9235	0.9223

\/ Orbital:   	d_xz	d_x2-y2	d_z2-r2	d_xy	d_yz
\/ Occupation:	0.8735	0.8554	0.4242	0.3141	0.222

##### U2-M1-disp0 #####
	d_xy		d_yz		d_xz		d_z2-r2		d_x2-y2		
d_xy	0.31 / 0.92	---- / ----	---- / ----	---- / ----	---- / ----	
d_yz	---- / ----	0.22 / 0.94	---- / ----	---- / ----	---- / ----	
d_xz	---- / ----	---- / ----	0.87 / 0.92	---- / ----	---- / ----	
d_z2-r2	---- / ----	---- / ----	---- / ----	0.42 / 0.92	0.16 / ----	
d_x2-y2	---- / ----	---- / ----	---- / ----	0.16 / ----	0.86 / 0.93	

/\ Orbita

In [4]:
### Evaluating energy surface with chg initialization in J-T state
#################################################
PREFIX = home + '/Projects/BaCoS2/correct_mag/disp-estruc/OUTCAR'

ITERS = [0, 5]
prefix = 'U2-G3-disp'

ITERS = [prefix + str(item) for item in ITERS]
print_occupations(ITERS, orbitals, order, atom_num=3)

##### U2-G3-disp0 #####
	d_xy		d_yz		d_xz		d_z2-r2		d_x2-y2		
d_xy	0.92 / 0.89	---- / ----	---- / ----	---- / -0.04	---- / ----	
d_yz	---- / ----	0.92 / 0.74	---- / ----	---- / ----	---- / ----	
d_xz	---- / ----	---- / ----	0.92 / 0.68	---- / ----	---- / ----	
d_z2-r2	---- / -0.04	---- / ----	---- / ----	0.93 / 0.23	---- / ----	
d_x2-y2	---- / ----	---- / ----	---- / ----	---- / ----	0.87 / 0.29	

/\ Orbital:   	d_z2-r2	d_xy	d_xz	d_yz	d_x2-y2
/\ Occupation:	0.9253	0.9248	0.9211	0.9196	0.8658

\/ Orbital:   	d_xy	d_yz	d_xz	d_x2-y2	d_z2-r2
\/ Occupation:	0.8894	0.7433	0.6766	0.2871	0.2316

##### U2-G3-disp5 #####
	d_xy		d_yz		d_xz		d_z2-r2		d_x2-y2		
d_xy	0.93 / 0.87	---- / ----	---- / ----	---- / -0.12	---- / -0.03	
d_yz	---- / ----	0.93 / 0.77	-0.01 / 0.23	---- / ----	---- / ----	
d_xz	---- / ----	-0.01 / 0.23	0.93 / 0.47	---- / ----	---- / ----	
d_z2-r2	---- / -0.12	---- / ----	---- / ----	0.93 / 0.28	---- / 0.05	
d_x2-y2	---- / -0.03	---- / ----	---- / ----	---- / 0.05	0.92 / 0.31	



In [3]:
### Evaluating energy surface with chg initialization in J-T state
#################################################
PREFIX = home + '/Projects/BaCoS2/correct_mag/disp-estruc/OUTCAR'

ITERS = [0, 6, 10]
prefix = 'chg-init-U2-M1-disp'

ITERS = [prefix + str(item) for item in ITERS]
print_occupations(ITERS, orbitals, order, atom_num=3)

##### chg-init-U2-M1-disp0 #####
	d_xy		d_yz		d_xz		d_z2-r2		d_x2-y2		
d_xy	0.94 / 0.84	---- / ----	---- / -0.03	---- / -0.2	---- / -0.02	
d_yz	---- / ----	0.93 / 0.88	---- / 0.1	---- / -0.02	---- / 0.01	
d_xz	---- / -0.03	---- / 0.1	0.94 / 0.29	---- / ----	---- / -0.01	
d_z2-r2	---- / -0.2	---- / -0.02	---- / ----	0.94 / 0.29	---- / ----	
d_x2-y2	---- / -0.02	---- / 0.01	---- / -0.01	---- / ----	0.94 / 0.29	

/\ Orbital:   	d_xz	d_x2-y2	d_z2-r2	d_xy	d_yz
/\ Occupation:	0.9437	0.938	0.9375	0.935	0.9335

\/ Orbital:   	d_yz	d_xy	d_xz	d_x2-y2	d_z2-r2
\/ Occupation:	0.8793	0.8443	0.2887	0.2885	0.286

##### chg-init-U2-M1-disp6 #####
	d_xy		d_yz		d_xz		d_z2-r2		d_x2-y2		
d_xy	0.93 / 0.88	---- / 0.05	---- / -0.06	---- / -0.1	---- / -0.04	
d_yz	---- / 0.05	0.94 / 0.74	---- / 0.24	---- / 0.05	---- / 0.02	
d_xz	---- / -0.06	---- / 0.24	0.94 / 0.49	---- / 0.04	---- / 0.02	
d_z2-r2	---- / -0.1	---- / 0.05	---- / 0.04	0.94 / 0.25	---- / 0.04	
d_x2-y2	---- / -0.04	---- / 0.02	---- / 0.02	---- / 0.

In [3]:
### Testing the two energy surfaces -- misaligned
#################################################
PREFIX = home + '/Projects/BaCoS2/correct_mag/disp-estruc/OUTCAR'

ITERS = [0, 1, 2, 5, 7.5]
prefix = 'U3-M1-disp'

ITERS = [prefix + str(item) for item in ITERS]
print_occupations(ITERS, orbitals, order, atom_num=3)

##### U3-M1-disp0 #####
	d_xy		d_yz		d_xz		d_z2-r2		d_x2-y2		
d_xy	0.94 / 0.92	---- / ----	---- / ----	---- / ----	---- / ----	
d_yz	---- / ----	0.95 / 0.57	---- / 0.35	---- / 0.01	---- / 0.01	
d_xz	---- / ----	---- / 0.35	0.95 / 0.57	---- / -0.01	---- / -0.01	
d_z2-r2	---- / ----	---- / 0.01	---- / -0.01	0.95 / 0.21	---- / 0.04	
d_x2-y2	---- / ----	---- / 0.01	---- / -0.01	---- / 0.04	0.96 / 0.26	

/\ Orbital:   	d_x2-y2	d_z2-r2	d_yz	d_xz	d_xy
/\ Occupation:	0.9576	0.9493	0.9474	0.9474	0.9413

\/ Orbital:   	d_xy	d_yz	d_xz	d_x2-y2	d_z2-r2
\/ Occupation:	0.9207	0.5692	0.5692	0.2589	0.2081

##### U3-M1-disp1 #####
	d_xy		d_yz		d_xz		d_z2-r2		d_x2-y2		
d_xy	0.94 / 0.92	---- / 0.01	---- / -0.01	---- / 0.01	---- / ----	
d_yz	---- / 0.01	0.95 / 0.57	---- / 0.34	---- / ----	---- / 0.02	
d_xz	---- / -0.01	---- / 0.34	0.95 / 0.59	---- / -0.03	---- / -0.01	
d_z2-r2	---- / 0.01	---- / ----	---- / -0.03	0.95 / 0.2	---- / 0.03	
d_x2-y2	---- / ----	---- / 0.02	---- / -0.01	---- / 0.03	0.96 / 0.26	


In [5]:
### Testing the two energy surfaces -- misaligned
#################################################
PREFIX = home + '/Projects/BaCoS2/correct_mag/disp-estruc/OUTCAR'

ITERS = [0, 5, 10]
prefix = 'U2-M1-disp'

ITERS = [prefix + str(item) for item in ITERS]
print_occupations(ITERS, orbitals, order, atom_num=3)

##### U2-M1-disp0 #####
	d_xy		d_yz		d_xz		d_z2-r2		d_x2-y2		
d_xy	0.93 / 0.9	---- / ----	---- / ----	---- / ----	---- / ----	
d_yz	---- / ----	0.94 / 0.61	---- / 0.26	---- / 0.01	---- / 0.01	
d_xz	---- / ----	---- / 0.26	0.94 / 0.61	---- / -0.01	---- / -0.01	
d_z2-r2	---- / ----	---- / 0.01	---- / -0.01	0.94 / 0.23	---- / 0.02	
d_x2-y2	---- / ----	---- / 0.01	---- / -0.01	---- / 0.02	0.93 / 0.28	

/\ Orbital:   	d_z2-r2	d_yz	d_xz	d_xy	d_x2-y2
/\ Occupation:	0.9359	0.9351	0.9351	0.9318	0.9262

\/ Orbital:   	d_xy	d_yz	d_xz	d_x2-y2	d_z2-r2
\/ Occupation:	0.9024	0.6118	0.6118	0.2797	0.2306

##### U2-M1-disp5 #####
	d_xy		d_yz		d_xz		d_z2-r2		d_x2-y2		
d_xy	0.93 / 0.9	---- / 0.02	---- / -0.02	---- / ----	---- / -0.03	
d_yz	---- / 0.02	0.93 / 0.6	---- / 0.28	---- / 0.01	---- / 0.02	
d_xz	---- / -0.02	---- / 0.28	0.93 / 0.64	---- / ----	---- / ----	
d_z2-r2	---- / ----	---- / 0.01	---- / ----	0.94 / 0.23	---- / 0.03	
d_x2-y2	---- / -0.03	---- / 0.02	---- / ----	---- / 0.03	0.92 / 0.28	

/\ 

In [4]:
### Testing the two energy surfaces -- misaligned
#################################################
PREFIX = home + '/Projects/BaCoS2/correct_mag/disp-estruc/OUTCAR'

ITERS = [0, 5, 10, 15, 20]
prefix = 'U3-G1-disp'

ITERS = [prefix + str(item) for item in ITERS]
print_occupations(ITERS, orbitals, order, atom_num=3)

##### U3-G1-disp0 #####
	d_xy		d_yz		d_xz		d_z2-r2		d_x2-y2		
d_xy	0.93 / 0.9	---- / ----	---- / ----	---- / 0.05	---- / ----	
d_yz	---- / ----	0.93 / 0.67	---- / ----	---- / ----	---- / ----	
d_xz	---- / ----	---- / ----	0.92 / 0.86	---- / ----	---- / ----	
d_z2-r2	---- / 0.05	---- / ----	---- / ----	0.94 / 0.17	---- / ----	
d_x2-y2	---- / ----	---- / ----	---- / ----	---- / ----	0.87 / 0.18	

/\ Orbital:   	d_z2-r2	d_xy	d_yz	d_xz	d_x2-y2
/\ Occupation:	0.94	0.9315	0.93	0.9193	0.867

\/ Orbital:   	d_xy	d_xz	d_yz	d_x2-y2	d_z2-r2
\/ Occupation:	0.8997	0.8566	0.6669	0.1825	0.1676

##### U3-G1-disp5 #####
	d_xy		d_yz		d_xz		d_z2-r2		d_x2-y2		
d_xy	0.93 / 0.9	---- / ----	---- / ----	---- / -0.02	---- / ----	
d_yz	---- / ----	0.92 / 0.8	---- / ----	---- / ----	---- / ----	
d_xz	---- / ----	---- / ----	0.92 / 0.75	---- / ----	---- / ----	
d_z2-r2	---- / -0.02	---- / ----	---- / ----	0.94 / 0.16	---- / ----	
d_x2-y2	---- / ----	---- / ----	---- / ----	---- / ----	0.87 / 0.18	

/\ Orbital:   

In [5]:
### Testing the two energy surfaces -- misaligned
#################################################
PREFIX = home + '/Projects/BaCoS2/correct_mag/disp-estruc/M-point/OUTCAR'

ITERS = [0, 1, 2, 5, 7.5]
prefix = 'U3-M1-disp'

ITERS = [prefix + str(item) for item in ITERS]
print_occupations(ITERS, orbitals, order, atom_num=3)

##### U3-M1-disp0 #####
	d_xy		d_yz		d_xz		d_z2-r2		d_x2-y2		
d_xy	0.94 / 0.92	---- / ----	---- / ----	---- / ----	---- / ----	
d_yz	---- / ----	0.95 / 0.57	---- / 0.35	---- / 0.01	---- / 0.01	
d_xz	---- / ----	---- / 0.35	0.95 / 0.57	---- / -0.01	---- / -0.01	
d_z2-r2	---- / ----	---- / 0.01	---- / -0.01	0.95 / 0.21	---- / 0.04	
d_x2-y2	---- / ----	---- / 0.01	---- / -0.01	---- / 0.04	0.96 / 0.26	

/\ Orbital:   	d_x2-y2	d_z2-r2	d_yz	d_xz	d_xy
/\ Occupation:	0.9575	0.9493	0.9474	0.9474	0.9413

\/ Orbital:   	d_xy	d_yz	d_xz	d_x2-y2	d_z2-r2
\/ Occupation:	0.9207	0.569	0.569	0.2596	0.2083

##### U3-M1-disp1 #####
	d_xy		d_yz		d_xz		d_z2-r2		d_x2-y2		
d_xy	0.94 / 0.92	---- / ----	---- / ----	---- / ----	---- / -0.01	
d_yz	---- / ----	0.95 / 0.57	---- / 0.35	---- / 0.01	---- / 0.01	
d_xz	---- / ----	---- / 0.35	0.95 / 0.58	---- / -0.01	---- / -0.01	
d_z2-r2	---- / ----	---- / 0.01	---- / -0.01	0.95 / 0.21	---- / 0.04	
d_x2-y2	---- / -0.01	---- / 0.01	---- / -0.01	---- / 0.04	0.96 / 0.26	



In [11]:
### Checking diagonalization -- rotated cell shows that the orbital ordering above is correct
### the yz and xz are indeed degenerate 
### and the t2g are in fact inverted

m = odms[5][0][Spin.down]
print(np.array(m))
eig = np.linalg.eig(m)
print()
print(np.round(eig[0],3))
print(np.round(eig[1],3))

NameError: name 'odms' is not defined

In [ ]:
m = odms[1][0][Spin.down]
print(m)
eig = np.linalg.eig(m)
print()
print(np.round(eig[0],2))
print(np.round(eig[1],2))